# Main Input

In [1]:
import findspark
findspark.init("/opt/spark")
from elasticsearch import Elasticsearch, helpers
import time
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
from pyspark.sql import SparkSession, functions as F

In [3]:
spark = (
    SparkSession.builder
    .appName("Spark Elasticsearch")
    .master("local[2]")
    .config("spark.driver.memory","2048m")
    .config("spark.sql.shuffle.partitions", 4) 
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") 
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.1,commons-httpclient:commons-httpclient:3.1,io.delta:delta-core_2.12:2.4.0") 
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
commons-httpclient#commons-httpclient added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-07697ec2-5b5f-46ab-b285-5603a014ba29;1.0
	confs: [default]
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.12.1 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in central
	found commons-logging#commons-logging;1.1.1 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-codec#commons-codec;1.2 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found o

In [5]:
df = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("delimiter", ";") \
    .load("market1mil.csv")

In [6]:
df.count()

999853

In [222]:
df.limit(5).toPandas()

,LOGICALREF,COUNT_,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,BRANCHNR,BRANCH,SALESMAN,CITY,REGION,LATITUDE,LONGITUDE,CLIENTCODE,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,SPECODE,CAPIBLOCK_CREADEDDATE
0,1,1,8,TOZ SEKER,15560,2.01.2017 00:00,45,"2,65","5,3","4,9",14,Batman Subesi,Gürsel ÜZÜM,Batman,Güneydoğu Anadolu,"37,8812","41,1351",6476,Hanım CANBULAT,None,None,İÇECEK,ÇAY KAHVE,SEKER TATLANDIRICI,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
1,2,1,20868,KIRMIZI MERCIMEK,15560,2.01.2017 00:00,"1,006","2,8","2,82","2,79",14,Batman Subesi,Gürsel ÜZÜM,Batman,Güneydoğu Anadolu,"37,8812","41,1351",6476,Hanım CANBULAT,167,BAKLİYAT,GIDA,BAKLİYAT,AÇIK BAKLİYAT,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
2,3,1,8583,"TEST MATIK 1,5 KG NORMAL",15560,2.01.2017 00:00,1,"4,95","4,95","4,19",14,Batman Subesi,Gürsel ÜZÜM,Batman,Güneydoğu Anadolu,"37,8812","41,1351",6476,Hanım CANBULAT,229,TEST,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,TOZ DETERJAN,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
3,4,1,1454,BIZIM MAKARNA BONCUK,15560,2.01.2017 00:00,1,"1,1","1,1","1,02",14,Batman Subesi,Gürsel ÜZÜM,Batman,Güneydoğu Anadolu,"37,8812","41,1351",6476,Hanım CANBULAT,146,ÜLKER,GIDA,MAKARNA,MAKARNA,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
4,5,1,13519,FILIZ MAKARNA KISA KESME 500 GR,15560,2.01.2017 00:00,1,"1,1","1,1","1,02",14,Batman Subesi,Gürsel ÜZÜM,Batman,Güneydoğu Anadolu,"37,8812","41,1351",6476,Hanım CANBULAT,52,FİLİZ,GIDA,MAKARNA,MAKARNA,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50


In [223]:
df.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- COUNT_: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- FICHENO: integer (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- LINENET: string (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- SALESMAN: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- CLIENTNAME: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (

# Clean

In [7]:
df1 = df.select("LOGICALREF", "ITEMCODE", "ITEMNAME", "AMOUNT", "PRICE", "LINENETTOTAL",
          "BRANCH", "CITY", "BRAND", "STARTDATE", "LATITUDE", "LONGITUDE")

In [8]:
df1 = df1.dropna()

In [9]:
df1 = df1.withColumn("original_datetime", to_timestamp(col("STARTDATE"), "dd.MM.yyyy HH:mm"))
df1 = df1.withColumn("STARTDATE", date_format(col("original_datetime"), "yyyy-MM-dd hh:mm"))
df1 = df1.withColumn("STARTDATE", (unix_timestamp(to_timestamp(col("STARTDATE"), "yyyy-MM-dd hh:mm")) * 1000))

In [10]:
# Replace commas with dots
df1 = df1.withColumn("LATITUDE", regexp_replace(col("LATITUDE"), ",", "."))
df1 = df1.withColumn("LONGITUDE", regexp_replace(col("LONGITUDE"), ",", "."))

# Cast to DoubleType while handling non-numeric values
df1 = df1.withColumn("LATITUDE", col("LATITUDE").cast(DoubleType()))
df1 = df1.withColumn("LONGITUDE", col("LONGITUDE").cast(DoubleType()))

df1 = df1.filter(
    (col("LATITUDE").between(-90, 90)) &
    (col("LONGITUDE").between(-180, 180))
)

# Create the LOCATION column
df1 = df1.withColumn("LOCATION", concat_ws(",", col("LATITUDE"), col("LONGITUDE")))

In [11]:
df1.count()

936156

In [228]:
df1.show(5)

+----------+--------+--------------------+------+-----+------------+-------------+------+--------+-------------+--------+---------+-------------------+---------------+
|LOGICALREF|ITEMCODE|            ITEMNAME|AMOUNT|PRICE|LINENETTOTAL|       BRANCH|  CITY|   BRAND|    STARTDATE|LATITUDE|LONGITUDE|  original_datetime|       LOCATION|
+----------+--------+--------------------+------+-----+------------+-------------+------+--------+-------------+--------+---------+-------------------+---------------+
|         2|   20868|    KIRMIZI MERCIMEK| 1,006|  2,8|        2,82|Batman Subesi|Batman|BAKLİYAT|1483435500000| 37.8812|  41.1351|2017-01-03 09:25:00|37.8812,41.1351|
|         3|    8583|TEST MATIK 1,5 KG...|     1| 4,95|        4,95|Batman Subesi|Batman|    TEST|1483435500000| 37.8812|  41.1351|2017-01-03 09:25:00|37.8812,41.1351|
|         4|    1454|BIZIM MAKARNA BONCUK|     1|  1,1|         1,1|Batman Subesi|Batman|   ÜLKER|1483435500000| 37.8812|  41.1351|2017-01-03 09:25:00|37.8812,4

In [229]:
df1 = df1.filter((col("AMOUNT") > 0) & 
                 (col("PRICE") > 0) & 
                 (col("LINENETTOTAL") > 0) & 
                 (col("LATITUDE") > 0) & 
                 (col("LONGITUDE") > 0))

df1 = df1.filter(col("STARTDATE").isNotNull())
df1 = df1.filter(col("LATITUDE").isNotNull())
df1 = df1.filter(col("LONGITUDE").isNotNull())

In [230]:
df1.limit(5).toPandas()

,LOGICALREF,ITEMCODE,ITEMNAME,AMOUNT,PRICE,LINENETTOTAL,BRANCH,CITY,BRAND,STARTDATE,LATITUDE,LONGITUDE,original_datetime,LOCATION
0,17,2123,TEKEL 2001 SOFT KISA,1,5,5,Adıyaman Subesi,Adıyaman,B.A.T TEKEL,1483435860000,37.7648,38.2786,2017-01-03 09:31:00,"37.7648,38.2786"
1,19,21713,L M KISA PAKET,1,7,7,Kırklareli Subesi,Kırklareli,PHILIP MORIS,1483436100000,41.7333,27.2167,2017-01-03 09:35:00,"41.7333,27.2167"
2,32,5917,CAMEL SOFT KISA 1913,1,8,8,İzmir Subesi,İzmir,JTİ,1483436640000,38.4189,27.1287,2017-01-03 09:44:00,"38.4189,27.1287"
3,36,5362,SİHİRLİ ELLER CİG KÖFTE 200GR,1,2,2,Afyonkarahisar Subesi,Afyonkarahisar,SİHİRLİ ELLER,1483437120000,38.7507,30.5567,2017-01-03 09:52:00,"38.7507,30.5567"
4,53,5229,LD BLUE UZUNPAKET,1,5,5,Malatya Subesi,Malatya,JTİ,1483438020000,38.3552,38.3095,2017-01-03 10:07:00,"38.3552,38.3095"


In [141]:
df1.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- STARTDATE: long (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- original_datetime: timestamp (nullable = true)
 |-- LOCATION: string (nullable = false)



# Write

In [231]:
df2 = df1.select("LOGICALREF", "ITEMCODE", "ITEMNAME", "AMOUNT", "PRICE", "LINENETTOTAL",
          "BRANCH", "CITY", "BRAND", "STARTDATE", "LOCATION")

In [232]:
online_markets = {
    "mappings": {
        "properties": {
            "LOGICALREF": {"type": "integer"},
            "ITEMCODE": {"type": "integer"},
            "ITEMNAME": {"type": "text"},
            "AMOUNT": {"type": "float"},
            "PRICE": {"type": "float"},
            "LINENETTOTAL": {"type": "float"},
            "BRANCH": {"type": "keyword"},
            "CITY": {"type": "keyword"},
            "BRAND": {"type": "keyword"},
            "STARTDATE": {"type": "date"},
            "LOCATION": {"type": "geo_point"} 
                    }
              }
              }

In [233]:
es = Elasticsearch("http://es:9200")

In [234]:
try:
    es.indices.delete("markets")
    print("markets  index deleted.")
except:
    print("No index")

markets  index deleted.


In [235]:
es.indices.create(index="markets", body=online_markets)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'markets'}

In [236]:
start_time = time.time()

df2.write \
    .format("org.elasticsearch.spark.sql") \
    .mode("append") \
    .option("es.nodes", "es") \
    .option("es.port","9200") \
    .save("markets")


print("----- %s secs -----" %(time.time() - start_time))

----- 18.34197187423706 secs -----


In [238]:
df_es = spark.read \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "es") \
    .option("es.port","9200") \
    .load("markets")

In [239]:
df_es.limit(5).toPandas()

,AMOUNT,BRANCH,BRAND,CITY,ITEMCODE,ITEMNAME,LINENETTOTAL,LOCATION,LOGICALREF,PRICE,STARTDATE
0,1.0,Samsun Subesi,B.A.T TEKEL,Samsun,16514,VICEROY RED KISA,6.0,"41.2928,36.3313",66257,6.0,2017-01-15 08:39:00
1,1.0,İstanbul Subesi,BİLLUR,İstanbul,14745,BILLUR TUZ 750 GR.,1.0,"41.0053,28.977",66259,1.0,2017-01-15 08:39:00
2,1.0,Şanlıurfa Subesi,ÜLKER,Şanlıurfa,245,ULKER SAKLIKOY KLASIK 140 GR,1.0,"37.1591,38.7969",66285,1.0,2017-01-15 08:45:00
3,1.0,Şanlıurfa Subesi,ÜLKER,Şanlıurfa,12210,ULKER SAKLIKOY SUTLU KREMALI 140 GR,1.0,"37.1591,38.7969",66286,1.0,2017-01-15 08:45:00
4,1.0,İstanbul Subesi,VİGOR,İstanbul,18921,HD KIRMIZI KALIN,5.0,"41.0053,28.977",66383,5.0,2017-01-15 00:30:00


In [240]:
df_es.printSchema()

root
 |-- AMOUNT: float (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- LINENETTOTAL: float (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- LOGICALREF: integer (nullable = true)
 |-- PRICE: float (nullable = true)
 |-- STARTDATE: timestamp (nullable = true)

